# Lowpass filter instead of Kalman filter for the PIT

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.plot import track_plot
from vessel_manoeuvring_models.equation import Equation

## Load models:
# (Uncomment these for faster loading):
#import vessel_manoeuvring_models.models.vmm_abkowitz  as vmm 
import vessel_manoeuvring_models.models.vmm_martin  as vmm 

In [ ]:
from vessel_manoeuvring_models.data import kalman_filter
from vessel_manoeuvring_models.data.lowpass_filter import lowpass_filter
import scipy.stats as stats

In [ ]:
id=22774
df, units, meta_data = mdl.load(id=id)
df_raw = df.copy()
df.index = df.index.total_seconds()
df.index-=df.index[0]

In [ ]:
def df_filter(df_gradient, cutoff=1, order=5):
    
    ts = np.mean(np.diff(df_gradient.index))
    fs = 1/ts
    
    df_lowpass = pd.DataFrame(columns=df_gradient.columns, index=df_gradient.index)
    for key,value in df_gradient.items():
        df_lowpass[key] = lowpass_filter(data=value, cutoff=cutoff, fs=fs, order=order)
        
    return df_lowpass

def lowpass(df_,cutoff=1, order=5):
    
    df_gradient = df_.copy()
    df_gradient.index=df.index
    
    df_gradient[['x0_filtered','y0_filtered','psi_filtered']] = df_filter(df_gradient[['x0','y0','psi']], cutoff=cutoff, order=order)
    df_gradient['x01d'] = np.gradient(df_gradient['x0_filtered'], df_gradient.index)
    df_gradient['y01d'] = np.gradient(df_gradient['y0_filtered'], df_gradient.index)
    df_gradient['z01d_gradient'] = np.gradient(df_gradient['z0'], df_gradient.index)
    
    df_gradient['r'] = df_gradient['psi1d'] = np.gradient(df_gradient['psi_filtered'], df_gradient.index)
    
    df_gradient[['x01d','y01d','r']] = df_filter(df_gradient[['x01d','y01d','r']], cutoff=cutoff, order=order)
    df_gradient['x02d'] = np.gradient(df_gradient['x01d'], df_gradient.index)
    df_gradient['y02d'] = np.gradient(df_gradient['y01d'], df_gradient.index)
    df_gradient['z02d_gradient'] = np.gradient(df_gradient['z01d_gradient'], df_gradient.index)
    df_gradient['r1d'] = df_gradient['psi2d'] = np.gradient(df_gradient['r'], df_gradient.index)
    
    df_gradient[['x02d','y02d','r1d']] = df_filter(df_gradient[['x02d','y02d','r1d']], cutoff=cutoff, order=order)
    
    
    return df_gradient

In [ ]:
df_lowpass = lowpass(df_=df, cutoff=1, order=1)

df_resample = df_raw.resample('0.5S').mean()
t_ = df_resample.index.total_seconds()
df_resample['x01d'] = np.gradient(df_resample['x0'], t_)
df_resample['y01d'] = np.gradient(df_resample['y0'], t_)
df_resample['r'] = np.gradient(df_resample['psi'], t_)
df_resample['x02d'] = np.gradient(df_resample['x01d'], t_)
df_resample['y02d'] = np.gradient(df_resample['y01d'], t_)
df_resample['r1d'] = np.gradient(df_resample['r'], t_)
df_resample.index=df_resample.index.total_seconds()
df_resample.index-=df_resample.index[0]


df_rolling = df.sort_index(ascending=False).rolling(100).mean().sort_index(ascending=True)
t_=df_rolling.index
df_rolling['x01d'] = np.gradient(df_rolling['x0'], t_)
df_rolling['y01d'] = np.gradient(df_rolling['y0'], t_)
df_rolling['r'] = np.gradient(df_rolling['psi'], t_)
df_rolling['x02d'] = np.gradient(df_rolling['x01d'], t_)
df_rolling['y02d'] = np.gradient(df_rolling['y01d'], t_)
df_rolling['r1d'] = np.gradient(df_rolling['r'], t_)

states = [
     ['x0','x01d','x02d'],
     ['y0','y01d','y02d'],
     ['psi','r','r1d']
]

for dof in states:
    fig,axes=plt.subplots(ncols=3)
    fig.set_size_inches(15,4)
    for state,ax in zip(dof,axes):
        ax.set_title(state)
        #df_lowpass_gradient.plot(y=state, ax=ax, label='lowpass gradient')
        df_lowpass.plot(y=state, ax=ax, label='lowpass')
        df_resample.plot(y=state, ax=ax, label='resample', alpha=0.5)
        df_rolling.plot(y=state, ax=ax, label='rolling', alpha=0.5)

In [ ]:
df = kalman_filter.transform_to_ship(df_lowpass, include_unfiltered=False)

df['thrust'] = df['Prop/PS/Thrust'] + df['Prop/SB/Thrust']
df['thrust'] = df_filter(df[['thrust']], cutoff=0.5, order=1)

df['U'] = np.sqrt(df['u']**2 + df['v']**2)


In [ ]:
df_cut = df.iloc[200:-200].copy()

In [ ]:
track_plot(df=df_cut, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', 
           beam=meta_data.beam);
        
df_cut.plot(y='delta')
df_cut.plot(y='x02d')
df_cut.plot(y='thrust')


In [ ]:
_, _, meta_data = mdl.load(id=id, dir_path='../data/processed/kalman_cut')
meta_data['rho']=1000
meta_data['mass'] = meta_data['Volume']*meta_data['rho']

# Ship parameters

In [ ]:
T_ = (meta_data.TA + meta_data.TF)/2
L_ = meta_data.lpp
m_ = meta_data.mass
rho_ = meta_data.rho
B_ = meta_data.beam
CB_ = m_/(T_*B_*L_*rho_)
I_z_ = m_*meta_data.KZZ**2
#I_z_ = 900


ship_parameters = {
        'T' : T_,
        'L' : L_,
        'CB' :CB_,
        'B' : B_,
        'rho' : rho_,
        'x_G' : 0,  # motions are expressed at CG
        'm' : m_,
        'I_z': I_z_, 
        'volume':meta_data.Volume,
    }

ps = prime_system.PrimeSystem(**ship_parameters)  # model

scale_factor = meta_data.scale_factor
ps_ship = prime_system.PrimeSystem(L=ship_parameters['L']*scale_factor, rho=meta_data['rho'])  # ship

# Brix parameters

In [ ]:
def calculate_prime(row, ship_parameters):
    return run(function=row['brix_lambda'], inputs=ship_parameters)


mask = df_parameters['brix_lambda'].notnull()
df_parameters.loc[mask,'brix_prime'] = df_parameters.loc[mask].apply(calculate_prime, ship_parameters=ship_parameters, axis=1)

df_parameters['brix_prime'].fillna(0, inplace=True)
#df_parameters['brix_SI'].fillna(0, inplace=True)



In [ ]:
df_parameters['prime'] = df_parameters['brix_prime']

def to_SI(row):
    
    prime = row['prime']
    denominator = row['denominator']
    lamda = lambdify(denominator)
    try:
        denominator_value = run(lamda, inputs=ship_parameters)
        SI = prime*denominator_value
    except:
        return None
    else:
        return SI
    
df_parameters['brix_SI'] = df_parameters.apply(to_SI, axis=1)
    

# Regression

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
def show_pred(X,y,results, label):
    
    display(results.summary())
    
    X_ = X
    y_ = y
    y_pred = results.predict(X_)
    
    prstd, iv_l, iv_u = wls_prediction_std(results, exog=X_, alpha=0.05)
    #iv_l*=-1 
    #iv_u*=-1
    
    fig,ax=plt.subplots()
    ax.plot(X_.index,y_, label='Numerical gradient from model test')
    ax.plot(X_.index,y_pred, '--', label='OLS')
    ax.set_ylabel(label)
    
    ax.fill_between(X_.index, y1=iv_l, y2=iv_u, zorder=-10, color='grey', alpha=0.5, label=r'5\% confidence')
    ax.legend();

## N

In [ ]:
N_ = sp.symbols('N_')

diff_eq_N = regression.DiffEqToMatrix(ode=vmm.N_qs_eq.subs(N_qs,N_), 
                                      label=N_, base_features=[delta,u,v,r])

In [ ]:
Math(vlatex(diff_eq_N.acceleration_equation))

In [ ]:
X = diff_eq_N.calculate_features(data=df_cut)
y = diff_eq_N.calculate_label(y=df_cut['r1d'])

model_N = sm.OLS(y,X)
results_N = model_N.fit()

show_pred(X=X,y=y,results=results_N, label=r'$\dot{r}$')

## Y

In [ ]:
Y_ = sp.symbols('Y_')
diff_eq_Y = regression.DiffEqToMatrix(ode=vmm.Y_qs_eq.subs(Y_qs,Y_), 
                                      label=Y_, base_features=[delta,u,v,r])

In [ ]:
Math(vlatex(diff_eq_Y.acceleration_equation))

In [ ]:
X = diff_eq_Y.calculate_features(data=df_cut)
y = diff_eq_Y.calculate_label(y=df_cut['v1d'])


model_Y = sm.OLS(y,X)
results_Y = model_Y.fit()

show_pred(X=X,y=y,results=results_Y, label=r'$\dot{v}$')

## X

In [ ]:
X_ = sp.symbols('X_')
diff_eq_X = regression.DiffEqToMatrix(ode=vmm.X_qs_eq.subs(X_qs,X_), 
                                      label=X_, base_features=[delta,u,v,r,thrust])

In [ ]:
diff_eq_X.acceleration_equation

In [ ]:
X = diff_eq_X.calculate_features(data=df_cut)
y = diff_eq_X.calculate_label(y=df_cut['u1d'])

model_X = sm.OLS(y,X)
results_X = model_X.fit()

show_pred(X=X,y=y,results=results_X, label=r'$\dot{u}}$')

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

In [ ]:
subs = {value:key for key,value in p.items()}
A_ = vmm.simulator.A*sp.matrices.MutableDenseMatrix([A_coeff,B_coeff,C_coeff])
A_lambda=lambdify(A_.subs(subs))

In [ ]:
A_coeff_ = results_summary_X['coeff']
B_coeff_ = results_summary_Y['coeff']
C_coeff_ = results_summary_N['coeff']

coeffs = run(A_lambda,A_coeff=A_coeff_.values, B_coeff=B_coeff_.values, C_coeff=C_coeff_.values, 
    **df_parameters['brix_SI'], **ship_parameters)

In [ ]:
results_summary_X['decoupled'] = coeffs[0][0]
results_summary_Y['decoupled'] = coeffs[1][0]
results_summary_N['decoupled'] = coeffs[2][0]


## Add the regressed parameters
Hydrodynamic derivatives that depend on acceleration cannot be obtained from the VCT regression. They are however essential if a time simulation should be conducted. These values have then been taken from Brix semi empirical formulas for the simulations below.

In [ ]:
df_parameters_all = df_parameters.copy()
for other in [results_summary_X, results_summary_Y, results_summary_N]:
    df_parameters_all = df_parameters_all.combine_first(other)

df_parameters_all.rename(columns={'decoupled':'regressed'}, inplace=True)
df_parameters_all.drop(columns=['brix_lambda'], inplace=True)

df_parameters_all['SI'] = df_parameters_all['regressed'].combine_first(df_parameters_all['brix_SI'])  # prefer regressed
df_parameters_all['SI'].fillna(0,inplace=True)

# Simulate forces

In [ ]:
df_cut['X_qs_pred'] = run(function=vmm.simulator.X_qs_lambda, inputs=df_cut, **df_parameters_all['SI'])
df_cut['Y_qs_pred'] = run(function=vmm.simulator.Y_qs_lambda, inputs=df_cut, **df_parameters_all['SI'])
df_cut['N_qs_pred'] = run(function=vmm.simulator.N_qs_lambda, inputs=df_cut, **df_parameters_all['SI'])


In [ ]:
X = diff_eq_X.calculate_features(data=df_cut)
X_forces = X*results_summary_X['coeff']
X_forces.index = df_cut.index

X = diff_eq_Y.calculate_features(data=df_cut)
Y_forces = X*results_summary_Y['coeff']
Y_forces.index = df_cut.index

X = diff_eq_N.calculate_features(data=df_cut)
N_forces = X*results_summary_N['coeff']
N_forces.index = df_cut.index


In [ ]:
display(px.line(X_forces, y=X_forces.columns, width=1000, height=400))
display(px.line(Y_forces, y=Y_forces.columns, width=1000, height=400))
display(px.line(N_forces, y=N_forces.columns, width=1000, height=400))

In [ ]:
df_captive = pd.DataFrame()

V_ = 2.0

betas = np.deg2rad(np.linspace(0,30,5))
deltas = np.deg2rad(np.linspace(-35,35,5))

Betas, Deltas = np.meshgrid(betas,deltas)

df_captive['beta'] = Betas.flatten()
df_captive['delta'] = Deltas.flatten()

df_captive['u'] = V_*np.cos(df_captive['beta'])
df_captive['v'] = -V_*np.sin(df_captive['beta'])
df_captive['thrust'] = 0
df_captive['r'] = 0


df_captive['X_qs_pred'] = run(function=vmm.simulator.X_qs_lambda, inputs=df_captive, **df_parameters_all['SI'])
df_captive['Y_qs_pred'] = run(function=vmm.simulator.Y_qs_lambda, inputs=df_captive, **df_parameters_all['SI'])
df_captive['N_qs_pred'] = run(function=vmm.simulator.N_qs_lambda, inputs=df_captive, **df_parameters_all['SI'])

keys = ['X_qs','Y_qs','N_qs']

x_keys = ['beta','delta']
for x_key in x_keys:
    
    fig,axes=plt.subplots(ncols=len(keys))
    fig.set_size_inches(15,4)
    
    others = list(x_keys)
    others.remove(x_key)
    
    for group_name, group in df_captive.groupby(others):
        
        if not isinstance(group_name,list):
            group_name = [group_name]
        label =''.join([f'{other}:{np.round(name,2)}, ' for other,name in zip(others,group_name)])
        
        for key,ax in zip(keys,axes):
            
            group.plot(x=x_key, y=f'{key}_pred',style='.-', ax=ax, label=label)
            #ax.get_legend().set_visible(False)
            ax.set_title(key)
            ax.grid(True)

# Simulation

The way that the regression is formulated, inertial forces, such as centrifugal force will be included into the derivatives (I think) which means that centrifugal force : $-m \cdot r \cdot u$ will be included into $Y_{ur}$ coefficient. This coefficient is therefore not pure hydrodynamic, and can potentially be counted twice..?
The coefficients are recalculated below to avooid this:

In [ ]:
parameters=df_parameters_all['SI'].copy()
x_G_ = ship_parameters['x_G']
#parameters['Xrr']+=(-m_*x_G_)
#parameters['Xvr']+=(-m_)
#parameters['Yur']+=m_
#parameters['Nur']+=m_*x_G_



In [ ]:
result_vmm = vmm.simulator.simulate(df_=df_cut, parameters=parameters, ship_parameters=ship_parameters)

In [ ]:
result_vmm.plot_compare()

In [ ]:
vmm.simulator.A

In [ ]:
S = sp.symbols('S')
S_eq=sp.Eq(S,(m-p.Yvdot)*(I_z-p.Nrdot) - (m*x_G-p.Yrdot)*(m*x_G-p.Nudot-p.Nvdot))
S_eq

In [ ]:
vmm.simulator.b

In [ ]:
A_inv = vmm.simulator.A.inv()

In [ ]:
inertia = sp.MutableDenseMatrix([-m*(-x_G*r**2-r*v), -m*r*u, -m*x_G*r*u])
accelerations = sp.MutableDenseMatrix([u1d, v1d, r1d])
sp.simplify(vmm.simulator.b - inertia)

In [ ]:
Math(vlatex(vmm.simulator.A*accelerations-inertia))

In [ ]:
Math(vlatex(eq_X_qs))

In [ ]:
Math(vlatex(vmm.simulator.A*accelerations-inertia))

In [ ]:
A_inv*(vmm.simulator.b-inertia) + A_inv*inertia

In [ ]:
A_inv*inertia

In [ ]:
solution = sp.simplify(A_inv*vmm.simulator.b)
solution

In [ ]:
solution[0]

In [ ]:
sp.fraction(solution[0])[0]